In [1]:
import sys

sys.argv = [""]
del sys

In [2]:
import argparse
import os
from copy import deepcopy

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

# causing the kernal to die.!!

from inpaint.data import PlacesDataset 
import matplotlib.pyplot as plt
import random

/Users/aadit/miniconda3/envs/inpaint/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [3]:
def flipCoin():
    f = random.random()
    return True if f < 0.5 else False

In [4]:
def create_cfg(args):
    parser = argparse.ArgumentParser()
    for key, val in args.items():
        parser.add_argument(key, default=val)

    cfg = parser.parse_args()
    return cfg

In [5]:
args = {}
# model
args["--in_channels"] = 4
args["--out_channels"] = 3
args["--latent_channels"] = 64
args["--pad_type"] = "zero"
args["--activation"] = "elu"
args["--norm_d"] = "none"
args["--norm_g"] = "none"
args["--init_type"] = "kaiming"
args["--init_gain"] = 0.02
args["--use_cuda"] = False

# dataset
args["--val_ds_dir"] = '../samples/Places365'
args["--crop_size"] = (256, 256)
args["--batch_size"] =  1
args["--num_workers"] = 1
args["--device_id"] = 0
args["--val_steps"] = 10

In [6]:
cfg = create_cfg(args)

In [7]:
val_ds = PlacesDataset(path_dir = cfg.val_ds_dir, transform_config=('to_tensor', 'center_crop', 'norm'), crop_size=cfg.crop_size)
val_loader = DataLoader(val_ds, batch_size=cfg.batch_size, shuffle=True, pin_memory=True, num_workers=cfg.num_workers)

/Users/aadit/miniconda3/envs/inpaint/lib/python3.8/site-packages/torchvision/transforms/transforms.py:852: UserWarning: Argument interpolation should be of type InterpolationMode instead of int. Please, use InterpolationMode enum.
  warnings.warn(


In [8]:
print(f"Total validation images: {len(val_ds)}")

Total validation images: 24


___

In [9]:
from inpaint.core.generator import GatedGenerator
from inpaint.tools import Evaluate
import glob

___

1. Initialize generator from a saved checkpoint

In [14]:
# load checkpoint and initialize
checkpoints_dir= '../../checkpoints/exp1'

checkpointslist= (glob.glob("../../checkpoints/exp1/*.pth")) 

# checkpointslist = os.listdir(checkpoints_dir)
print(checkpointslist)

# best_model_index = checkpointslist.index('../../checkpoints/exp1/best_models_final.pth')
best_model_index = checkpointslist.index('../../checkpoints/exp1/ckpt_epoch1.pth')


best_model = checkpointslist[best_model_index]
# print(best_model)

generator = GatedGenerator(cfg)

models = torch.load(best_model, map_location=torch.device('cpu') )
gen_model_state_dict = models["generator_state_dict"]
generator.load_state_dict(gen_model_state_dict)


['../../checkpoints/exp1/best_models_final.pth', '../../checkpoints/exp1/ckpt_epoch1.pth', '../../checkpoints/exp1/ckpt_epoch30.pth']


<All keys matched successfully>

___

2. Create Evaluate instance pass dataloader and generator instance to Evaluate 

In [15]:
eval_gen = Evaluate(generator,val_loader)

____
3. Evaluate (return avg psnr and avg ssim value

In [16]:
avg_psnr, avg_ssim = eval_gen.evaluate()

/Users/aadit/miniconda3/envs/inpaint/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: 
  warn(f"Failed to load image Python extension: {e}")


In [17]:
print(avg_psnr, avg_ssim)

68.98837693718744 0.8659873157739639


In [ ]:
# best model : avg_psnr= 71.03444448747706 avg_ssim = 0.8908961191773415

In [ ]:
# checkpoint 1 : avg_psnr = 68.98837693718744 avg_ssim = 0.8659873157739639